In [8]:
import requests
import pandas as pd

# Define the API URL and headers
url = "https://api.nike.com/discover/product_wall/v1/marketplace/US/language/en/consumerChannelId/d9a5bc42-4b9c-4976-858a-f159cf99c647"
headers = {
    "nike-api-caller-id": "nike:dotcom:browse:wall.client:2.0",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": "\"Windows\"",
    "Referer": "https://www.nike.com/",
    "sec-ch-ua": "\"Google Chrome\";v=\"131\", \"Chromium\";v=\"131\", \"Not_A Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0"
}

# Pagination parameters
count = 24  # Number of records per request
anchor = 0  # Starting index
all_products_data = []  # List to store all product data

while True:
    # Define query parameters for the current page
    params = {
        "path": "/w/mens-shoes-nik1zy7ok",
        "attributeIds": "16633190-45e5-4830-a068-232ac7aea82c,0f64ecc7-d624-4e91-b171-b83a03dd8550",
        "queryType": "PRODUCTS",
        "anchor": anchor,
        "count": count
    }

    # Make the API request
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()

        # Check if "productGroupings" is valid
        if not data.get("productGroupings"):
            print(f"No more product groupings found. Exiting at anchor {anchor}.")
            break

        # Extract product data
        for grouping in data["productGroupings"]:
            for product in grouping.get("products", []):
                product_details = {
                    "Title": product.get("copy", {}).get("title", "N/A"),
                    "Subtitle": product.get("copy", {}).get("subTitle", "N/A"),
                    "Price": product.get("prices", {}).get("currentPrice", "N/A"),
                    "Initial Price": product.get("prices", {}).get("initialPrice", "N/A"),
                    "Discount (%)": product.get("prices", {}).get("discountPercentage", "N/A"),
                    "Color": product.get("displayColors", {}).get("simpleColor", {}).get("label", "N/A"),
                    "Color Description": product.get("displayColors", {}).get("colorDescription", "N/A"),
                    "Image URL": product.get("colorwayImages", {}).get("portraitURL", "N/A"),
                    "Product URL": product.get("pdpUrl", {}).get("url", "N/A"),
                    "Badge": product.get("badgeLabel", "N/A")
                }
                all_products_data.append(product_details)

        # Update the anchor for the next page
        anchor += count

        # Check if we've reached the total resources
        total_resources = data.get("pages", {}).get("totalResources", 0)
        if anchor >= total_resources:
            print("All products have been fetched.")
            break
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print(response.text)
        break

# Create a pandas DataFrame
df = pd.DataFrame(all_products_data)

# Save the DataFrame to a CSV file
output_file = "nike_products_all.csv"
df.to_csv(output_file, index=False)
print(f"Fetched {len(df)} records and saved to {output_file}")


All products have been fetched.
Fetched 2709 records and saved to nike_products_all.csv


In [7]:
print(data)

{'searchTerm': None, 'canonicalUrl': None, 'title': None, 'primaryHeading': None, 'attributeIds': ['16633190-45e5-4830-a068-232ac7aea82c', '0f64ecc7-d624-4e91-b171-b83a03dd8550'], 'consumerChannelId': 'd9a5bc42-4b9c-4976-858a-f159cf99c647', 'marketplace': 'US', 'locationId': None, 'language': 'en', 'selectedFiltersCount': None, 'mlModels': ['NONE'], 'analyzer': None, 'contentData': None, 'navigation': None, 'productGroupings': None, 'pages': {'next': '', 'prev': '/discover/product_wall/v1/marketplace/US/language/en/consumerChannelId/d9a5bc42-4b9c-4976-858a-f159cf99c647?path=/w/mens-shoes-nik1zy7ok&attributeIds=16633190-45e5-4830-a068-232ac7aea82c,0f64ecc7-d624-4e91-b171-b83a03dd8550&queryType=PRODUCTS&anchor=792&count=24', 'totalPages': 34, 'totalResources': 799}}


In [10]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "nike_products_all.csv"  # Replace with your file name
df = pd.read_csv(file_path)

# Add synthetic features
# 1. Discount Amount
df["Discount Amount"] = df["Initial Price"] - df["Price"]

# 2. Discount Flag
df["Discount Flag"] = (df["Discount (%)"] > 0).astype(int)

# 3. Price Category (Low: <50, Medium: 50-100, High: >100)
df["Price Category"] = pd.cut(df["Price"], bins=[0, 50, 100, np.inf], labels=["Low", "Medium", "High"])

# 4. Color Diversity Score (Count the number of colors in 'Color Description')
df["Color Diversity Score"] = df["Color Description"].apply(lambda x: len(x.split("/")))

# 5. Badge Encoding
badge_types = df["Badge"].unique()
for badge in badge_types:
    if pd.notna(badge):
        df[f"Is {badge}"] = (df["Badge"] == badge).astype(int)

# Feature Engineering
# 6. One-hot encode 'Color'
df = pd.concat([df, pd.get_dummies(df["Color"], prefix="Color")], axis=1)

# 7. Label encode 'Price Category'
df["Price Category"] = df["Price Category"].cat.codes

# 8. Extract dominant color from 'Color Description'
df["Dominant Color"] = df["Color Description"].apply(lambda x: x.split("/")[0] if pd.notna(x) else "Unknown")

# 9. NLP features from 'Title' and 'Subtitle' (Word Count)
df["Title Word Count"] = df["Title"].apply(lambda x: len(str(x).split()))
df["Subtitle Word Count"] = df["Subtitle"].apply(lambda x: len(str(x).split()))

# 10. Marketing Appeal Score (Badge and Discount combined)
df["Marketing Appeal Score"] = df["Discount (%)"] * df["Discount Flag"]

# Save the modified dataset
processed_file_path = "nike_products_with_features.csv"
df.to_csv(processed_file_path, index=False)
print(f"Processed dataset saved to {processed_file_path}")

# Preview the processed data
print(df.head())


Processed dataset saved to nike_products_with_features.csv
       Title     Subtitle  Price  Initial Price  Discount (%)   Color  \
0  Nike C1TY        Shoes  100.0            100             0    Pink   
1  Nike C1TY        Shoes  100.0            100             0    Grey   
2  Nike C1TY  Men's Shoes  100.0            100             0  Orange   
3  Nike C1TY        Shoes  100.0            100             0   Brown   
4  Nike C1TY        Shoes  100.0            100             0   Brown   

                                   Color Description    Badge  \
0      Red Stardust/Canyon Rust/Burgundy Crush/Black  Just In   
1   Smoke Grey/Medium Ash/Aquarius Blue/Summit White  Just In   
2       Safety Orange/Cosmic Clay/Black/Summit White  Just In   
3  Desert Khaki/Parachute Beige/University Gold/B...  Just In   
4             Soft Pearl/Light Khaki/Lightning/Black      NaN   

   Discount Amount  Discount Flag  ...  Color_Orange  Color_Pink  \
0              0.0              0  ...     